In [1]:
from swat import *
import swat as sw
from pprint import pprint
%matplotlib inline
import matplotlib
import sys

# sys.path.append(r'\\sashq\root\dept\cas\weshiz\github_vb025_dev\python-dlpy')
sys.path.append('C:\\Users\\weshiz\\Documents\\GitHub\\modify\\python-dlpy')
sys.path.append('C:\\Users\\weshiz\\Documents\\GitHub\\python-fcmp')
from dlpy.layers import * 
from dlpy.applications import *
from dlpy import Model, Sequential
from dlpy.utils import *
from dlpy.splitting import two_way_split
from dlpy.images import *
from dlpy.model import *
from src.parser import *
from src.decorator import *
from dlpy.lr_scheduler import *

In [2]:
s = sw.CAS('dlgrd009', 13300)

### Load Image

In [3]:
img_path='/disk/linux/dlpy/Giraffe_Dolphin'
img_path='/cas/DeepLearn/data/Giraffe_Dolphin'
my_images = ImageTable.load_files(s, path=img_path)
my_images.resize(112)

### Cyclic Learning Rate Policy

In [4]:
batch_size = 64
factor = 2
num_batch_per_epoch = math.ceil(s.numrows(my_images).numrows / batch_size)
step_size = int(num_batch_per_epoch * factor)

In [5]:
num_batch_per_epoch, step_size

(7, 14)

In [6]:
def cyclic_lr(rate, iterNum, batch, initRate):
    num_batch_per_epoch = 10
    step_size = 10
    max_lr = 0.01
    batch_cum = num_batch_per_epoch * iterNum + batch
    cycle = int(batch_cum / (2 * step_size) + 1)
    x = abs(batch_cum / step_size - 2 * cycle + 1)
    rate = initRate + (max_lr - initRate) * max(0, 1-x)
    return rate

In [8]:
fcmp_code = python_to_fcmp(func=cyclic_lr, print=True)

('function cyclic_lr(rate, iterNum, batch, initRate);\n'
 '    num_batch_per_epoch = 10;\n'
 '    step_size = 10;\n'
 '    max_lr = 0.01;\n'
 '    batch_cum = (num_batch_per_epoch * iterNum + batch);\n'
 '    cycle = int((batch_cum / 2 * step_size + 1));\n'
 '    x = abs(((batch_cum / step_size - 2 * cycle) + 1));\n'
 '    rate = (initRate + (max_lr - initRate) * max(0, (1 - x)));\n'
 '    return (rate);\n'
 'endsub;\n')


In [12]:
register_fcmp_routines(conn=s, routine_code=fcmp_code, function_tbl_name='cyclic_lr')

NOTE: Cloud Analytic Services saved the file CYCLIC_LR.sashdat in caslib CASUSER(weshiz).


In [13]:
lr_scheduler = FCMPLR(conn=s, fcmp_learning_rate='cyclic_lr', learning_rate=0.0001)
solver = MomentumSolver(lr_scheduler=lr_scheduler,
                        clip_grad_max=100, clip_grad_min=-100)
optimizer = Optimizer(algorithm=solver, mini_batch_size=16, log_level=3, max_epochs=50, reg_l2=0.0005)
gpu = Gpu(devices=[0])

The following argument(s) learning_rate, gamma, step_size, power are overwritten by the according arguments specified in lr_scheduler.


### Build Model

In [9]:
def conv_block(x, filters, size, stride=1, mode='same', act=True):
    x = Conv2d(filters, size, size, act='identity', include_bias=False, stride=stride)(x)
    x = BN(act='relu' if act else 'identity')(x)
    return x

In [10]:
def res_block(ip, nf=64):
    x = conv_block(ip, nf, 3, 2)
    x = conv_block(x, nf, 3, 1, act=False)
    return Res()([x, ip])

In [11]:
inp_resnet= Input(3, 112, 112, scale = 1.0 / 255, name='InputLayer_1')
x=conv_block(inp_resnet, 64, 9, 1)
for i in range(4): x=res_block(x)
x=Conv2d(20, 9, 9, act='tanh')(x)
x=Pooling(7, 7)(x)
output = OutputLayer(n=2)(x)
resnet_like_model = Model(s, inputs = inp_resnet, outputs = output)
resnet_like_model.compile()

NOTE: Model compiled successfully.


### Train

In [14]:
resnet_like_model.fit(data=my_images, n_threads=4, record_seed=13309, optimizer=optimizer,
                      gpu=gpu, log_level=2)

NOTE: Inputs=_image_ is used
NOTE: Training from scratch.
NOTE: Using dlgrd009.unx.sas.com: 1 out of 4 available GPU devices.
NOTE:  Synchronous mode is enabled.
NOTE:  The total number of parameters is 415358.
NOTE:  The approximate memory cost is 115.00 MB.
NOTE:  Loading weights cost       0.00 (s).
NOTE:  Initializing each layer cost       2.17 (s).
NOTE:  The total number of threads on each worker is 4.
NOTE:  The total mini-batch size per thread on each worker is 16.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 64.
NOTE:  Target variable: _label_
NOTE:  Number of levels for the target variable:      2
NOTE:  Levels for the target variable:
NOTE:  Level      0: Dolphin
NOTE:  Level      1: Giraffe
NOTE:  Number of input variables:     1
NOTE:  Number of numeric input variables:      1
NOTE:  Batch nUsed Learning Rate        Loss  Fit Error     L2Norm   Time(s) (Training)
NOTE:      0    64   0.0001           0.8488        0.5     0.2999     0.5

NOTE:      1    64   0.0001           0.6359     0.3438     0.2998     0.02
NOTE:      2    64   0.0001           0.6733     0.3906     0.2998     0.02
NOTE:      3    64   0.0001           0.6614      0.375     0.2998     0.02
NOTE:      4    64   0.0001           0.6829     0.4063     0.2998     0.02
NOTE:      5    64   0.0001           0.6844     0.4063     0.2998     0.02
NOTE:      6    64   0.0001           0.6134     0.3125     0.2998     0.02
NOTE:  Epoch Learning Rate        Loss  Fit Error   Time(s)
NOTE:  10       0.0001           0.669     0.3862     0.16
NOTE:  Batch nUsed Learning Rate        Loss  Fit Error     L2Norm   Time(s) (Training)
NOTE:      0    64   0.0001           0.6912     0.4219     0.2998     0.03
NOTE:      1    64   0.0001           0.6904     0.4219     0.2998     0.02
NOTE:      2    64   0.0001           0.7589     0.5156     0.2998     0.02
NOTE:      3    64   0.0001           0.7016     0.4375     0.2998     0.02
NOTE:      4    64   0.0001      

NOTE:      4    64   0.0001           0.6818        0.5     0.2998     0.02
NOTE:      5    64   0.0001           0.6573     0.4531     0.2998     0.02
NOTE:      6    64   0.0001             0.63     0.3906     0.2998     0.02
NOTE:  Epoch Learning Rate        Loss  Fit Error   Time(s)
NOTE:  22       0.0001          0.6558     0.4353     0.17
NOTE:  Batch nUsed Learning Rate        Loss  Fit Error     L2Norm   Time(s) (Training)
NOTE:      0    64   0.0001           0.6839     0.5156     0.2998     0.03
NOTE:      1    64   0.0001           0.6289     0.3906     0.2998     0.02
NOTE:      2    64   0.0001           0.6693     0.4688     0.2998     0.02
NOTE:      3    64   0.0001           0.6965     0.5156     0.2998     0.02
NOTE:      4    64   0.0001           0.6743     0.4688     0.2998     0.02
NOTE:      5    64   0.0001           0.6594     0.4531     0.2998     0.02
NOTE:      6    64   0.0001           0.6336     0.4063     0.2998     0.02
NOTE:  Epoch Learning Rate       

NOTE:      1    64   0.0001           0.4976       0.25     0.2997     0.02
NOTE:      2    64   0.0001           0.4788     0.1875     0.2997     0.02
NOTE:      3    64   0.0001           0.4897     0.2031     0.2997     0.02
NOTE:      4    64   0.0001           0.5567     0.3438     0.2997     0.02
NOTE:      5    64   0.0001           0.4893     0.1719     0.2997     0.02
NOTE:      6    64   0.0001           0.4751      0.125     0.2997     0.02
NOTE:  Epoch Learning Rate        Loss  Fit Error   Time(s)
NOTE:  37       0.0001          0.4988     0.2165     0.17
NOTE:  Batch nUsed Learning Rate        Loss  Fit Error     L2Norm   Time(s) (Training)
NOTE:      0    64   0.0001           0.4702     0.1719     0.2997     0.03
NOTE:      1    64   0.0001           0.4984     0.2188     0.2997     0.02
NOTE:      2    64   0.0001           0.4909     0.2344     0.2997     0.02
NOTE:      3    64   0.0001           0.4605     0.1094     0.2997     0.02
NOTE:      4    64   0.0001      

,Descr,Value
0,Model Name,model_hisnor
1,Model Type,Convolutional Neural Network
2,Number of Layers,26
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,10
6,Number of Pooling Layers,1
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,9
9,Number of Residual Layers,4


### reduce_lr_on_plateau

In [15]:
@cast_array('loss')
def reduce_lr_on_plateau(rate, initRate, gamma, loss):
    cool_down_iters = 5
    patience = 1
    len_ = len(loss)
    temp_rate = initRate
    cool_down_counter = cool_down_iters
    best = loss[0]
    for i in range(len_):
        if loss[i] < best:
            best = loss[i]
            bad_epoch = 0
        else:
            bad_epoch = bad_epoch + 1;

        if cool_down_counter > 0:
            cool_down_counter = cool_down_counter - 1
            bad_epoch = 0

        if bad_epoch > patience:
            temp_rate = temp_rate * gamma
            cool_down_counter = cool_down_iters
            bad_epoch = 0
    rate = temp_rate
    return rate

Arguments, loss, are casted to array type


In [16]:
rlr_fcmp_code = python_to_fcmp(func=reduce_lr_on_plateau, print=True)

('function reduce_lr_on_plateau(rate, initRate, gamma, loss[*]);\n'
 '    cool_down_iters = 5;\n'
 '    patience = 1;\n'
 '    len_ = dim(loss);\n'
 '    temp_rate = initRate;\n'
 '    cool_down_counter = cool_down_iters;\n'
 '    best = loss[1];\n'
 '    do i = 0 to len_ by 1;\n'
 '        if loss[i + 1] < best then do;\n'
 '            best = loss[i + 1];\n'
 '            bad_epoch = 0;\n'
 '        end;\n'
 '        else do;\n'
 '            bad_epoch = (bad_epoch + 1);\n'
 '        end;\n'
 '        if cool_down_counter > 0 then do;\n'
 '            cool_down_counter = (cool_down_counter - 1);\n'
 '            bad_epoch = 0;\n'
 '        end;\n'
 '        if bad_epoch > patience then do;\n'
 '            temp_rate = temp_rate * gamma;\n'
 '            cool_down_counter = cool_down_iters;\n'
 '            bad_epoch = 0;\n'
 '        end;\n'
 '    end;\n'
 '    rate = temp_rate;\n'
 '    return (rate);\n'
 'endsub;\n')


In [17]:
register_fcmp_routines(conn=s, routine_code=rlr_fcmp_code, function_tbl_name='reduce_lr_on_plateau')

NOTE: Cloud Analytic Services saved the file REDUCE_LR_ON_PLATEAU.sashdat in caslib CASUSER(weshiz).


In [18]:
lr_scheduler = FCMPLR(conn=s, fcmp_learning_rate='reduce_lr_on_plateau', learning_rate=0.0001)
solver = MomentumSolver(lr_scheduler=lr_scheduler,
                        clip_grad_max=100, clip_grad_min=-100)
optimizer = Optimizer(algorithm=solver, mini_batch_size=16, log_level=3, max_epochs=50, reg_l2=0.0005)
gpu = Gpu(devices=[0])

The following argument(s) learning_rate, gamma, step_size, power are overwritten by the according arguments specified in lr_scheduler.


In [19]:
resnet_like_model.fit(data=my_images, n_threads=4, record_seed=13309, optimizer=optimizer,
                      gpu=gpu, log_level=2)

NOTE: Inputs=_image_ is used
NOTE: Training based on existing weights.
NOTE: Using dlgrd009.unx.sas.com: 1 out of 4 available GPU devices.
NOTE:  Synchronous mode is enabled.
NOTE:  The total number of parameters is 415358.
NOTE:  The approximate memory cost is 115.00 MB.
NOTE:  Loading weights cost       0.00 (s).
NOTE:  Initializing each layer cost       1.68 (s).
NOTE:  The total number of threads on each worker is 4.
NOTE:  The total mini-batch size per thread on each worker is 16.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 64.
NOTE:  Target variable: _label_
NOTE:  Number of levels for the target variable:      2
NOTE:  Levels for the target variable:
NOTE:  Level      0: Dolphin
NOTE:  Level      1: Giraffe
NOTE:  Number of input variables:     1
NOTE:  Number of numeric input variables:      1
NOTE:  Batch nUsed Learning Rate        Loss  Fit Error     L2Norm   Time(s) (Training)
NOTE:      0    64   0.0001           0.3391     0.0625     0

NOTE:      4    64   0.0001            0.258    0.03125     0.2997     0.02
NOTE:      5    64   0.0001           0.2609    0.03125     0.2997     0.02
NOTE:      6    64   0.0001           0.2743    0.03125     0.2997     0.03
NOTE:  Epoch Learning Rate        Loss  Fit Error   Time(s)
NOTE:  11       0.0001          0.2648    0.02902     0.17
NOTE:  Batch nUsed Learning Rate        Loss  Fit Error     L2Norm   Time(s) (Training)
NOTE:      0    64   0.0001           0.2355          0     0.2997     0.03
NOTE:      1    64   0.0001           0.2651    0.03125     0.2997     0.02
NOTE:      2    64   0.0001           0.2666    0.03125     0.2997     0.02
NOTE:      3    64   0.0001           0.2767    0.04688     0.2997     0.02
NOTE:      4    64   0.0001           0.2561    0.01563     0.2997     0.02
NOTE:      5    64   0.0001           0.2329    0.03125     0.2997     0.02
NOTE:      6    64   0.0001           0.2528    0.03125     0.2997     0.02
NOTE:  Epoch Learning Rate       

NOTE:      5    64  0.00001           0.2008    0.04688     0.2996     0.02
NOTE:      6    64  0.00001           0.2311    0.01563     0.2996     0.02
NOTE:  Epoch Learning Rate        Loss  Fit Error   Time(s)
NOTE:  24         1E-5          0.2181    0.04018     0.17
NOTE:  Batch nUsed Learning Rate        Loss  Fit Error     L2Norm   Time(s) (Training)
NOTE:      0    64  0.00001           0.2355    0.01563     0.2996     0.03
NOTE:      1    64  0.00001           0.1971          0     0.2996     0.02
NOTE:      2    64  0.00001           0.2343    0.01563     0.2996     0.02
NOTE:      3    64  0.00001           0.1906    0.01563     0.2996     0.02
NOTE:      4    64  0.00001           0.1971          0     0.2996     0.02
NOTE:      5    64  0.00001           0.2436    0.03125     0.2996     0.02
NOTE:      6    64  0.00001           0.2249    0.01563     0.2996     0.02
NOTE:  Epoch Learning Rate        Loss  Fit Error   Time(s)
NOTE:  25         1E-5          0.2176    0.01339

NOTE:  Batch nUsed Learning Rate        Loss  Fit Error     L2Norm   Time(s) (Training)
NOTE:      0    64     1E-7           0.2075    0.01563     0.2996     0.03
NOTE:      1    64     1E-7           0.2463     0.0625     0.2996     0.02
NOTE:      2    64     1E-7           0.2241    0.01563     0.2996     0.02
NOTE:      3    64     1E-7           0.1992          0     0.2996     0.02
NOTE:      4    64     1E-7           0.2448     0.0625     0.2996     0.02
NOTE:      5    64     1E-7           0.1865          0     0.2996     0.02
NOTE:      6    64     1E-7           0.2164    0.01563     0.2996     0.02
NOTE:  Epoch Learning Rate        Loss  Fit Error   Time(s)
NOTE:  38         1E-7          0.2178    0.02455     0.16
NOTE:  Batch nUsed Learning Rate        Loss  Fit Error     L2Norm   Time(s) (Training)
NOTE:      0    64     1E-7           0.2307    0.04688     0.2996     0.03
NOTE:      1    64     1E-7           0.2485    0.04688     0.2996     0.02
NOTE:      2    64   

,Descr,Value
0,Model Name,model_hisnor
1,Model Type,Convolutional Neural Network
2,Number of Layers,26
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,10
6,Number of Pooling Layers,1
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,9
9,Number of Residual Layers,4


In [20]:
s.endsession()

+ Elapsed: 0.000123s, user: 0.000101s, mem: 0.203mb